In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
import sqlite3

In [ ]:
os.chdir(r'/content/drive/MyDrive/AppliedAI/self_case_study_1/ashrae_energy_prediction')

In [ ]:
class controller():
    def __init__(self):
        self.building_id_list = list(range(0,1449))
        self.year_upper_limit = 2018
        self.year_lower_limit = 2016
        self.service = service()

    def predict(self, building_id, year):
        try:
            assert isinstance(building_id, int), 'building id should be integer'
            assert building_id in self.building_id_list, 'please provide valid building id'
            assert isinstance(year, int), 'year should be integer'
            assert (year >= self.year_lower_limit) & (year <= self.year_upper_limit),\
                   f'year should within {(self.year_lower_limit, self.year_upper_limit)}'
        except AssertionError as e:
            return e
        output = self.service.predict(building_id, year)
        return output

class dao():
    def __init__(self):
        self.conn = sqlite3.connect('data/input_data/ashrae.db')

    def get_meter_list(self, building_id):
        query = """
                SELECT meter FROM building_meter_map
                WHERE building_id==?;
                """
        cursor = self.conn.execute(query, (building_id,))
        self.conn.commit()
        return (cursor.fetchall())

    def get_preprocessed_data(self, building_id, year, meter, config):
        feature = config['feature']

        
        feature_str = config['feature_str']
        query = """
                    SELECT 
                        {}
                    FROM building_metadata
                    INNER JOIN weather_data ON building_metadata.site_id==weather_data.site_id
                    WHERE building_metadata.building_id==? AND strftime('%Y', weather_data.timestamp)==?
                    ORDER BY weather_data.timestamp;
                """.format(feature_str)
        cursor = self.conn.execute(query, (building_id,str(year)))
        df = pd.DataFrame(cursor.fetchall(), columns=feature)
        self.conn.commit()
        return df

class service():
    def __init__(self):
        self.model_0_1 = pickle.load(open('model/model_0_1.pkl', 'rb'))
        self.model_0_2 = pickle.load(open('model/model_0_2.pkl', 'rb'))
        self.model_1 = pickle.load(open('model/model_1.pkl', 'rb'))
        self.model_2 = pickle.load(open('model/model_2.pkl', 'rb'))
        self.model_3 = pickle.load(open('model/model_3.pkl', 'rb'))
        self.model_0_config = pickle.load(open('model/model_0_config.pkl', 'rb'))
        self.model_1_config = pickle.load(open('model/model_1_config.pkl', 'rb'))
        self.model_2_config = pickle.load(open('model/model_2_config.pkl', 'rb'))
        self.model_3_config = pickle.load(open('model/model_3_config.pkl', 'rb'))
        self.building_sample_1 = pickle.load(open('model/building_sample_1.pkl', 'rb'))
        self.building_sample_2 = pickle.load(open('model/building_sample_2.pkl', 'rb'))
        self.dao = dao()

    def post_processing(self, x):
        return np.clip(np.expm1(x), 0, None)

    def predict(self, building_id, year):
        #load data
        meter_list = self.dao.get_meter_list(building_id)
        output = dict()
        for meter_type in meter_list:
            meter = meter_type[0]
            #get_processed_data
            if meter == 0:
                preprocessed_data = self.dao.get_preprocessed_data(building_id, year, meter,
                                                                   self.model_0_config)
                if building_id in self.building_sample_1:
                    output[meter] = self.post_processing(self.model_0_1.predict(preprocessed_data))
                else:
                    output[meter] = self.post_processing(self.model_0_2.predict(preprocessed_data))
            elif meter == 1:
                preprocessed_data = self.dao.get_preprocessed_data(building_id, year, meter,
                                                                   self.model_1_config)
                output[meter] = self.post_processing(self.model_1.predict(preprocessed_data))
            elif meter == 2:
                preprocessed_data = self.dao.get_preprocessed_data(building_id, year, meter,
                                                                   self.model_2_config)
                output[meter] = self.post_processing(self.model_2.predict(preprocessed_data))
            elif meter == 3:
                preprocessed_data = self.dao.get_preprocessed_data(building_id, year, meter,
                                                                   self.model_3_config)
                output[meter] = self.post_processing(self.model_3.predict(preprocessed_data))
        return output

In [ ]:
model_0_config = {'feature':['air_temperature', 
                            'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
                            'air_temperature_max_past_18',
                            'air_temperature_min_past_18', 'air_temperature_mean_past_18',
                            'air_temperature_median_past_18', 'air_temperature_std_past_18',
                            'cloud_coverage_max_past_18', 'cloud_coverage_min_past_18',
                            'cloud_coverage_mean_past_18', 'cloud_coverage_median_past_18',
                            'cloud_coverage_std_past_18', 'dew_temperature_max_past_18',
                            'dew_temperature_min_past_18', 'dew_temperature_mean_past_18',
                            'dew_temperature_median_past_18', 'dew_temperature_std_past_18',
                            'sin_hour', 'sin_month', 
                            'meter', 'building_id', 'quarter',
                            'site_id', 'primary_use', 'floor_count',
                            'log_sq_ft', 'month',
                            'dayofweek', 'hour'],
                  'feature_str': f"""air_temperature, 
                                     dew_temperature, precip_depth_1_hr, sea_level_pressure,
                                     air_temperature_max_past_18,
                                     air_temperature_min_past_18, air_temperature_mean_past_18,
                                     air_temperature_median_past_18, air_temperature_std_past_18,
                                     cloud_coverage_max_past_18, cloud_coverage_min_past_18,
                                     cloud_coverage_mean_past_18, cloud_coverage_median_past_18,
                                     cloud_coverage_std_past_18, dew_temperature_max_past_18,
                                     dew_temperature_min_past_18, dew_temperature_mean_past_18,
                                     dew_temperature_median_past_18, dew_temperature_std_past_18,
                                     sin_hour, sin_month, 0 AS meter, building_id, quarter,
                                     building_metadata.site_id, primary_use, floor_count,
                                     log_sq_ft, month, dayofweek, hour
                                    """}
model_1_config = {'feature':['air_temperature', 
                            'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
                            'air_temperature_max_past_18',
                            'air_temperature_min_past_18', 'air_temperature_mean_past_18',
                            'air_temperature_median_past_18', 'air_temperature_std_past_18',
                            'cloud_coverage_max_past_18', 'cloud_coverage_min_past_18',
                            'cloud_coverage_mean_past_18', 'cloud_coverage_median_past_18',
                            'cloud_coverage_std_past_18', 'dew_temperature_max_past_18',
                            'dew_temperature_min_past_18', 'dew_temperature_mean_past_18',
                            'dew_temperature_median_past_18', 'dew_temperature_std_past_18',
                            'sin_hour', 'sin_month', 
                            'meter', 'building_id', 'quarter',
                            'site_id', 'primary_use', 'floor_count',
                            'log_sq_ft', 'month',
                            'dayofweek', 'hour'],
                  'feature_str': f"""air_temperature, 
                                     dew_temperature, precip_depth_1_hr, sea_level_pressure,
                                     air_temperature_max_past_18,
                                     air_temperature_min_past_18, air_temperature_mean_past_18,
                                     air_temperature_median_past_18, air_temperature_std_past_18,
                                     cloud_coverage_max_past_18, cloud_coverage_min_past_18,
                                     cloud_coverage_mean_past_18, cloud_coverage_median_past_18,
                                     cloud_coverage_std_past_18, dew_temperature_max_past_18,
                                     dew_temperature_min_past_18, dew_temperature_mean_past_18,
                                     dew_temperature_median_past_18, dew_temperature_std_past_18,
                                     sin_hour, sin_month, 0 AS meter, building_id, quarter,
                                     building_metadata.site_id, primary_use, floor_count,
                                     log_sq_ft, month, dayofweek, hour
                                    """}
model_2_config = {'feature':['air_temperature', 
                            'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
                            'air_temperature_max_past_18',
                            'air_temperature_min_past_18', 'air_temperature_mean_past_18',
                            'air_temperature_median_past_18', 'air_temperature_std_past_18',
                            'cloud_coverage_max_past_18', 'cloud_coverage_min_past_18',
                            'cloud_coverage_mean_past_18', 'cloud_coverage_median_past_18',
                            'cloud_coverage_std_past_18', 'dew_temperature_max_past_18',
                            'dew_temperature_min_past_18', 'dew_temperature_mean_past_18',
                            'dew_temperature_median_past_18', 'dew_temperature_std_past_18',
                            'sin_hour', 'sin_month', 
                            'meter', 'building_id', 'quarter',
                            'site_id', 'primary_use', 'floor_count',
                            'log_sq_ft',
                            'dayofweek', 'hour'],
                  'feature_str': f"""air_temperature, 
                                     dew_temperature, precip_depth_1_hr, sea_level_pressure,
                                     air_temperature_max_past_18,
                                     air_temperature_min_past_18, air_temperature_mean_past_18,
                                     air_temperature_median_past_18, air_temperature_std_past_18,
                                     cloud_coverage_max_past_18, cloud_coverage_min_past_18,
                                     cloud_coverage_mean_past_18, cloud_coverage_median_past_18,
                                     cloud_coverage_std_past_18, dew_temperature_max_past_18,
                                     dew_temperature_min_past_18, dew_temperature_mean_past_18,
                                     dew_temperature_median_past_18, dew_temperature_std_past_18,
                                     sin_hour, sin_month, 0 AS meter, building_id, quarter,
                                     building_metadata.site_id, primary_use, floor_count,
                                     log_sq_ft, dayofweek, hour
                                    """}
model_3_config = {'feature':['air_temperature', 
                            'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
                            'air_temperature_max_past_18',
                            'air_temperature_min_past_18', 'air_temperature_mean_past_18',
                            'air_temperature_median_past_18', 'air_temperature_std_past_18',
                            'cloud_coverage_max_past_18', 'cloud_coverage_min_past_18',
                            'cloud_coverage_mean_past_18', 'cloud_coverage_median_past_18',
                            'cloud_coverage_std_past_18', 'dew_temperature_max_past_18',
                            'dew_temperature_min_past_18', 'dew_temperature_mean_past_18',
                            'dew_temperature_median_past_18', 'dew_temperature_std_past_18',
                            'sin_hour', 'sin_month', 
                            'meter', 'building_id', 'quarter',
                            'site_id', 'primary_use', 'floor_count',
                            'log_sq_ft',
                            'dayofweek', 'hour'],
                  'feature_str': f"""air_temperature, 
                                     dew_temperature, precip_depth_1_hr, sea_level_pressure,
                                     air_temperature_max_past_18,
                                     air_temperature_min_past_18, air_temperature_mean_past_18,
                                     air_temperature_median_past_18, air_temperature_std_past_18,
                                     cloud_coverage_max_past_18, cloud_coverage_min_past_18,
                                     cloud_coverage_mean_past_18, cloud_coverage_median_past_18,
                                     cloud_coverage_std_past_18, dew_temperature_max_past_18,
                                     dew_temperature_min_past_18, dew_temperature_mean_past_18,
                                     dew_temperature_median_past_18, dew_temperature_std_past_18,
                                     sin_hour, sin_month, 0 AS meter, building_id, quarter,
                                     building_metadata.site_id, primary_use, floor_count,
                                     log_sq_ft, dayofweek, hour
                                    """}
pickle.dump(model_0_config, open('model/model_0_config.pkl', 'wb'))
pickle.dump(model_1_config, open('model/model_1_config.pkl', 'wb'))
pickle.dump(model_2_config, open('model/model_2_config.pkl', 'wb'))
pickle.dump(model_3_config, open('model/model_3_config.pkl', 'wb'))